In [4]:
import cv2
import os
import mediapipe as mp

# Force CPU usage to avoid GPU-related crashes
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Initialize MediaPipe solutions
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_face = mp.solutions.face_mesh

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

# Set camera resolution (optional but recommended)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Initialize holistic model
with mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=1,
        smooth_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            # Convert the BGR image to RGB
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process the image and get the result
            results = holistic.process(img)

            # Convert back to BGR for OpenCV
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            # Draw face landmarks
            if results.face_landmarks:
                mp_drawing.draw_landmarks(
                    img, results.face_landmarks, mp_face.FACEMESH_TESSELATION)

            # Draw left hand landmarks
            if results.left_hand_landmarks:
                mp_drawing.draw_landmarks(
                    img, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Draw right hand landmarks
            if results.right_hand_landmarks:
                mp_drawing.draw_landmarks(
                    img, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Draw pose landmarks
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Show the final output
            cv2.imshow('MediaPipe Holistic', img)

            # Break the loop on pressing 'q'
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    except Exception as e:
        print("An error occurred:", e)

    finally:
        cap.release()
        cv2.destroyAllWindows()


In [3]:
import cv2
import mediapipe as mp
import time

# Initialize MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize variables
expression = ""
result_shown = False
last_input_time = 0
input_delay = 1.5  # seconds to wait before allowing new input

def count_fingers(hand_landmarks):
    tips = [4, 8, 12, 16, 20]
    count = 0
    for tip in tips[1:]:
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y:
            count += 1
    return count

def detect_operator(hand_landmarks):
    # Only thumb up: +
    if (hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x and 
        all(hand_landmarks.landmark[tip].y > hand_landmarks.landmark[tip - 2].y for tip in [8, 12, 16, 20])):
        return '+'
    
    # Only index up: -
    if (hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y and 
        all(hand_landmarks.landmark[tip].y > hand_landmarks.landmark[tip - 2].y for tip in [12, 16, 20])):
        return '-'

    # Victory sign: *
    if (hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y and 
        hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y and 
        all(hand_landmarks.landmark[tip].y > hand_landmarks.landmark[tip - 2].y for tip in [16, 20])):
        return '*'

    # All fingers up: /
    if all(hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y for tip in [8, 12, 16, 20]):
        return '/'
    
    return None

# Start webcam and hand detection
cap = cv2.VideoCapture(0)
with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)

        current_time = time.time()

        if result.multi_hand_landmarks:
            hand_landmarks = result.multi_hand_landmarks[0]
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Detect operator
            op = detect_operator(hand_landmarks)
            if op and current_time - last_input_time > input_delay:
                expression += op
                last_input_time = current_time
                result_shown = False

            # Detect number
            elif not op and current_time - last_input_time > input_delay:
                fingers = count_fingers(hand_landmarks)
                if fingers <= 5:
                    expression += str(fingers)
                    last_input_time = current_time

        # Keyboard inputs
        key = cv2.waitKey(10)
        if key == ord('r') or key == ord('='):
            try:
                result = str(eval(expression))
                expression += "=" + result
                result_shown = True
            except:
                expression = "Error"
                result_shown = True
        elif key == ord('c'):
            expression = ""
            result_shown = False
        elif key == ord('q'):
            break

        # Display expression
        cv2.rectangle(image, (0, 0), (640, 50), (0, 0, 0), -1)
        cv2.putText(image, expression, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
        cv2.imshow("Gesture Calculator", image)

cap.release()
cv2.destroyAllWindows()
